# Introduction to the Polars GPU Engine

<img src="https://raw.githubusercontent.com/rapidsai-community/tutorial/refs/heads/main/images/polars-benchmark-best-queries.png" style="float: right; margin-left: 5px; width: 500px;">

Polars is a popular single machine DataFrame library powered by an OLAP Query Engine. Beginning in the v1.3 release, 
Polars can now leverage NVIDIA GPUs for even higher performance through its GPU engine (powered by RAPIDS cuDF).

Designed to make processing 10-100+ GBs of data feel interactive with just a single GPU, this new engine is built 
directly into the Polars Lazy API – just pass `**engine="gpu"` to the `collect` operation.

The GPU engine fully utilizes the Polars optimizer to ensure efficient execution and minimal memory usage, is compatible 
with the ecosystem of tools built for Polars, and has graceful CPU fallback for unsupported queries.

**Note:** The GPU engine is only be available in Python Polars.

This notebook is a short introduction to the Polars GPU engine, powered by cuDF. If you want to learn more about the 
implementation details, check out the [GPU engine release blog](https://pola.rs/posts/gpu-engine-release/) by the Polars
team. 

**Attribution:** This notebook was adapted from [the polars GPU engine demo](https://github.com/rapidsai-community/showcase/blob/main/accelerated_data_processing_examples/polars_gpu_engine_demo.ipynb) 
from the rapids community repo. 

## The Polars GPU Engine

- If you are running this on Google Colab, you are all set. 
- If you follow the local install setup via conda, you are also set. 
- If you installed dependencies via pip, you will need to do: 

```bash
pip install - U polars[gpu] --extra-index-url=https://pypi.nvidia.com
```

## Data 
We'll be working with a sample (20%) of the dataset of [simulated financial transactions from Kaggle](https://www.kaggle.com/datasets/conorsully1/simulated-transactions)

If you are running this locally, and you followed the steps in the [0.Welcome_and_Setup.ipynb](https://github.com/rapidsai-community/tutorial/blob/main/0.Welcome_and_Setup.ipynb) notebook, you should have the `/data` folder ready to go. 

### Google Colab Instructions

In the next step we download a script that will allow you to get the data for this notebook session.


In [ ]:

# colab: uncomment next line to get the data setup script
#! wget https://raw.githubusercontent.com/rapidsai-community/tutorial/refs/heads/main/data_setup.py


In [ ]:
# colab: uncomment next line to get the pageviews data set
#! python data_setup.py --transactions

In [ ]:
# Verify that you are running with an NVIDIA GPU
! nvidia-smi

To begin, let's use Polars to read the parquet file and look at the schema and first few rows.

With this dataset, we expect to see some meaningful performance benefits using NVIDIA GPUs for computationally heavy 
queries and limited benefits for basic queries that are primarily bottlenecked by reading data.

In [ ]:
import polars as pl
pl.__version__

In [ ]:
transactions = pl.scan_parquet("../data/transactions.parquet")

In [ ]:
transactions.collect_schema()

In [ ]:
transactions.head(5).collect()

This dataset has a mix of data types (strings, dates, integers, and floats), some missing values, and it looks like the 
dates were actually parsed as strings. We'll need to handle that during some of our analysis.

## Let's use GPUs: Total aggregate transaction amount

With Polars, to calculate the total spending of all transactions we simply `sum` the `AMOUNT` column:

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

Looks like we're handling a high total transaction volume. Let's run the same query on the GPU.

To use a GPU, we just need to tell Polars to use the GPU engine when we call `collect` by passing `engine="gpu"` as a parameter.


```python
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")
```

The default configuration is likely the right choice for most workloads, but sometimes we want more control. We can 
provide more detailed configuration options (such as which device to run on and a variety of other options) by passing a
Polars `GPUEngine` object to the `engine` parameter instead.


In this notebook, we'll use `pl.GPUEngine`. The default configuration has transparent CPU fallback for unsupported 
operations, so if we execute an unsupported query we don't get an error. To prove we're running on the GPU, we'll pass a
configured engine object that will raise an error if we can't run the query.

_If you're running with [jupyterlab-nvdashboard](https://developer.nvidia.com/blog/maximize-gpu-performance-with-near-real-time-usage-stats-on-nvdashboard-v0-10/) you should see the GPU Memory and Utilization tick up._

In [ ]:
gpu_engine = pl.GPUEngine(
    device=0,  # This is the default
    raise_on_fail=True,  # Fail loudly if we can't run on the GPU.
)

The very first collection on the GPU will take a couple of seconds. The GPU engine is lazy-loaded so that even if the 
necessary packages are installed, Polars' fast import times are not affected. Consequently, when we trigger GPU execution
for the first time, we load a number of additional packages, and initialize GPU-specific data structures and contexts.

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

We probably don't need a GPU for such a simple operation on this dataset. But when we start doing more complex analysis, 
the high-bandwidth memory and compute power of GPUs will make things much more interactive.

## More Complex Queries

While the data is synthetic, it's representative of the kinds of datasets that come up in financial services, retail/e-commerce,
consumer internet, and other industries.

With this data, we can see how using GPU-accelerated Polars provides significant productivity boosts by exploring common
business questions

## Which customers have the largest total transactions?

In [ ]:
%%time
res_cpu = (
    transactions.group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

Since the dataset is fairly small, the execution of the query takes only a few seconds, but let's see what we get 
by running the same query on a GPU. 

To achieve this, is as simple as passing the `gpu_engine` we set above in our `.collect()` call:

In [ ]:
%%time

res_gpu = (
    transactions.group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

Great! We see a nice performance gain when using the GPU engine!

**Exercise:** Find the average transaction amount for each expense type.

<details>
  <summary>Solution (click dropdown) </summary>
  <p>

```python
# to run this type it in a code cell
ex_res_gpu = (
    transactions.group_by("EXP_TYPE")
    .agg(pl.col("AMOUNT").mean())
    .sort(by="AMOUNT", descending=True)
    .collect(engine=gpu_engine)
)
ex_res_gpu
```
  </p>
</details>



In [ ]:
# your solution here


## What about Polars SQL? 

In addition to the Dataframe interface, Polars also has an SQL interface. We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.


In [ ]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt DESC
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

## Which customers have the largest single transaction?

Customer `CP2KXQSX9I` had the largest total transactions over time (on the full dataset), but they might not have the largest single transaction. Let's find the top customers by single transaction amount.

In [ ]:
%%time

(
    transactions.group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

In [ ]:
%%time

(
    transactions.group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

Again, we see a nice speedup using the GPU engine.

**Exercise:** Find the average transaction amount for each expense type.

_Hints_:
1. First, think about what columns you need to group by to get daily transactions
   - You'll need YEAR, MONTH, and DAY columns
   - Remember that these are separate columns in the dataset

2. For each day, we want to know:
   - How many transactions occurred (len)
   - The average transaction amount (mean)

3. The aggregation should include:
   - A count of all transactions
   - The mean of the AMOUNT column
   - Use .alias() to give meaningful names to the results

4. After getting the results:
   - Sort by transaction count in descending order
   - Take only the top 10 busiest days

<details>
  <summary>Solution (click dropdown) </summary>
  <p>

```python
# to run this type it in a code cell
active_days_gpu = (
    transactions.group_by(["YEAR", "MONTH", "DAY"])
    .agg([
        pl.len().alias("transaction_count"),
        pl.col("AMOUNT").mean().alias("avg_amount")
    ])
    .sort(by="transaction_count", descending=True)
    .head(10)
    .collect(engine=gpu_engine)
)
active_days_gpu
```
  </p>
</details>


In [ ]:
# your solution here

## Investigating a specific customer

Let's explore the transactions of a single customer now. What was the largest transaction for customer `CIP0I11MG2`?

In [ ]:
%%time

(
    transactions.filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

In [ ]:
%%time

(
    transactions.filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect(engine=gpu_engine)
)

### Why was performance using the CPU and GPU engines similar for this query?

What we've seen so far is that, for many common queries, we see significant performance increases when we use the GPU. 
But for this last one, things were more similar.

Using Polars on a GPU isn't always a surefire winner in terms of speed when compared to Polars on a CPU. For simple queries
that aren't computationally heavy, like the last query, we're often limited by the speed that we can read results from disk.

Let's confirm this by profiling.

In [ ]:
res, prof = (
    transactions.filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100).alias(
        "pct_time_spent"
    )
)

We spent 99.9%+ of the time just reading the data. Polars can use the GPU-accelerated Parquet reader to read this data, but ultimately when we're **IO bound** like this, there's less opportunity for GPU-acceleration.

Let's try an even more computationally intense query.

## What's the per-month transaction amount for each category over time?

For this query, we'll group and sort down to the individual month, which takes more work.

In [ ]:
%%time
res = (
    transactions.group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

In [ ]:
%%time
res = (
    transactions.group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

Again, since this query does more work we see strong performance benefits on the GPU.

## Conclusion

With the new Polars GPU engine powered by cuDF, you can potentially achieve significant performance gains and lower costs
for workflows processing 10-100+ GB of data.

Key takeaways from this notebook:
- The Polars GPU engine seamlessly integrates with cuDF to accelerate DataFrame operations
- For IO-bound operations like reading data, GPU acceleration benefits are limited  
- Computationally intensive operations like grouping and sorting show significant speedups on GPU
- Switching between CPU and GPU engines requires minimal code changes - just add `engine=gpu_engine`

To learn more, we encourage you to visit the [Polars GPU support documentation](https://docs.rapids.ai/api/cudf/stable/cudf_polars/) 
or visit [rapids.ai/cudf-polars](rapids.ai/cudf-polars).

In the next notebook, we will learn about the accelerating machine learning workflows with cuML:

[Next Notebook: 4 Intro to cuML →](https://colab.research.google.com/github/rapidsai-community/tutorial/blob/main/4.Intro_to_cuML.ipynb)
